# Aplicativo para Detecção de Deepfakes - Streamlit

Neste notebook, para ser possível fazer as detecções, é necessário que na aba 'Ambiente de execução', 'Alterar o tipo de ambinete de execução', esteja selecionado o modo com alguma GPU.

**Para ter acesso a alguns exemplos para testar o APP, clone o repositório a seguir e use as imagens e vídeos da pasta 'samples'**

In [1]:
# Clonar o repositório do projeto TCC
!git clone https://github.com/NathFarinha/TCC_DeepFake_Detection_v1.git

Cloning into 'TCC_DeepFake_Detection_v1'...
remote: Enumerating objects: 375, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 375 (delta 45), reused 144 (delta 21), pack-reused 196
Receiving objects: 100% (375/375), 98.14 MiB | 39.36 MiB/s, done.
Resolving deltas: 100% (105/105), done.


**Organizando ambiente para execução do APP**

In [2]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [3]:
# Clonar o repositório do projeto do artigo 'Video Face Manipulation Detection Through Ensemble of CNNs'
!git clone https://github.com/polimi-ispl/icpr2020dfdc
!pip install efficientnet-pytorch
!pip install -U git+https://github.com/albu/albumentations > /dev/null
%cd icpr2020dfdc

Cloning into 'icpr2020dfdc'...
remote: Enumerating objects: 645, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 645 (delta 96), reused 80 (delta 80), pack-reused 537
Receiving objects: 100% (645/645), 99.63 MiB | 26.09 MiB/s, done.
Resolving deltas: 100% (336/336), done.
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=10ee9645fd4667a8769c7e4fa85e79ab26c8ceccbda2e233433d35793046a608
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch
  Running command git clone --filter=blob:none --quiet https://github.com/albu/albumentations /tmp/pip-req-build-stqm8o10
/content/icpr2020dfdc


**Criando o APP Streamlit**

In [4]:
%%writefile app.py
import torch
from torch.utils.model_zoo import load_url
from scipy.special import expit
from PIL import Image
import streamlit as st
import os
import tempfile

from blazeface import FaceExtractor, BlazeFace, VideoReader
from architectures import fornet, weights
from isplutils import utils

# Configuração do dispositivo
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 32

# Inicialize o modelo de detecção facial BlazeFace
facedet = BlazeFace().to(device)
facedet.load_weights("blazeface/blazeface.pth")
facedet.load_anchors("blazeface/anchors.npy")
videoreader = VideoReader(verbose=False)
video_read_fn = lambda x: videoreader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn=video_read_fn, facedet=facedet)

# Função para realizar a detecção de deep fakes com base no modelo selecionado
def detect_deep_fake(uploaded_file, selected_model, selected_dataset):
    # Crie um diretório temporário para salvar o arquivo
    temp_dir = tempfile.mkdtemp()

    # Salve o arquivo temporariamente no diretório temporário
    temp_file_path = os.path.join(temp_dir, uploaded_file.name)
    with open(temp_file_path, 'wb') as temp_file:
        temp_file.write(uploaded_file.read())

    if uploaded_file.type.startswith('image'):
        im = Image.open(temp_file_path)
        im_faces = face_extractor.process_image(img=im)
        im_face = im_faces['faces'][0] if len(im_faces['faces']) > 0 else None

        model_url = weights.weight_url['{:s}_{:s}'.format(selected_model, selected_dataset)]
        net = getattr(fornet, selected_model)().eval().to(device)
        net.load_state_dict(load_url(model_url, map_location=device, check_hash=True))
        transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

        if im_face is not None:
            faces_t = torch.stack([transf(image=im_face)['image']])

            with torch.no_grad():
                faces_pred = torch.sigmoid(net(faces_t.to(device))).cpu().numpy().flatten()

            avg_score = expit(faces_pred.mean())
            prediction = 'FAKE' if avg_score >= 0.6 else 'REAL'
        else:
            return 'Não foi possível detectar uma face na imagem.'

    elif uploaded_file.type.startswith('video'):
        model_url = weights.weight_url['{:s}_{:s}'.format(selected_model, selected_dataset)]
        net = getattr(fornet, selected_model)().eval().to(device)
        net.load_state_dict(load_url(model_url, map_location=device, check_hash=True))
        transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

        vid_faces = face_extractor.process_video(temp_file_path)
        faces_t = torch.stack([transf(image=frame['faces'][0])['image'] for frame in vid_faces if len(frame['faces'])])

        with torch.no_grad():
            faces_pred = net(faces_t.to(device)).cpu().numpy().flatten()

        avg_score = expit(faces_pred.mean())
        prediction = 'FAKE' if avg_score >= 0.6 else 'REAL'

    else:
        return 'Tipo de arquivo não suportado.'

    return prediction, avg_score

# Configuração de estilo do Streamlit
st.set_page_config(
    page_title="Detecção de Deep Fakes",
    page_icon="✅",
    layout="wide"
)

# Página de detecção de Deep Fakes
page = st.sidebar.radio("Selecione uma página", ["Detecção de Deep Fakes", "Informações da Autora"])

if page == "Detecção de Deep Fakes":
    # Cabeçalho do aplicativo Streamlit
    st.title('Detecção de Deep Fakes')

    # Upload de arquivo de imagem ou vídeo
    uploaded_file = st.file_uploader('Envie uma imagem ou vídeo', type=['jpg', 'jpeg', 'png', 'mp4'])

    if uploaded_file:
        selected_model = st.selectbox('Selecione o modelo', ['EfficientNetB4', 'EfficientNetB4ST','EfficientNetAutoAttB4','EfficientNetAutoAttB4ST'])  # Substitua pelos modelos disponíveis
        selected_dataset = st.selectbox('Selecione o conjunto de dados', ['DFDC', 'FFPP'])  # Substitua pelos conjuntos de dados disponíveis

        if st.button('Detecção'):
            prediction, avg_score = detect_deep_fake(uploaded_file, selected_model, selected_dataset)

            if uploaded_file.type.startswith('image'):
                st.image(uploaded_file, caption='Imagem enviada', width=500)
            elif uploaded_file.type.startswith('video'):
                st.video(uploaded_file, format='video/mp4')

            st.subheader(prediction)
            avg_score = "{:.4f}".format(avg_score)
            st.write(f'Pontuação média: {avg_score}')


elif page == "Informações da Autora":
    st.title('Informações da Autora')
    st.markdown("### Nome:")
    st.write("Nathalia Farinha Rodrigues")
    st.markdown("### TCC:")
    st.write("ANÁLISE DE MODELOS DETECTORES DE DEEPFAKE USANDO APRENDIZADO PROFUNDO")
    st.markdown("### Curso:")
    st.write("Engenharia Elétrica")
    st.markdown("### Orientador:")
    st.write("Prof. Dr. Frank Herman Behrens")
    st.markdown("### Coorientador:")
    st.write("Prof. Dr. Ademar Takeo Akabane")
    st.markdown("### Ano:")
    st.write("Campinas 2023")


Writing app.py


**Execute os próximos passos para acessar o link do APP no streamlit**

In [5]:
! ls -la # verifica arquivos
! cat app.py # visualizar conteudo do arquivo

total 188
drwxr-xr-x 10 root root  4096 Oct  1 15:49 .
drwxr-xr-x  1 root root  4096 Oct  1 15:48 ..
-rw-r--r--  1 root root  5210 Oct  1 15:49 app.py
drwxr-xr-x  3 root root  4096 Oct  1 15:49 architectures
drwxr-xr-x  2 root root  4096 Oct  1 15:49 assets
drwxr-xr-x  2 root root  4096 Oct  1 15:49 blazeface
-rw-r--r--  1 root root   357 Oct  1 15:49 environment.yml
-rw-r--r--  1 root root 16943 Oct  1 15:49 extract_faces.py
drwxr-xr-x  8 root root  4096 Oct  1 15:49 .git
-rw-r--r--  1 root root    50 Oct  1 15:49 .gitignore
-rw-r--r--  1 root root  3271 Oct  1 15:49 index_celebdf.py
-rw-r--r--  1 root root  3323 Oct  1 15:49 index_dfdc.py
-rw-r--r--  1 root root  3724 Oct  1 15:49 index_ffpp.py
drwxr-xr-x  2 root root  4096 Oct  1 15:49 isplutils
-rw-r--r--  1 root root 35149 Oct  1 15:49 LICENSE
drwxr-xr-x  4 root root  4096 Oct  1 15:49 notebook
-rw-r--r--  1 root root  7761 Oct  1 15:49 README.md
drwxr-xr-x  2 root root  4096 Oct  1 15:49 scripts
drwxr-xr-x  3 root root  4096 Oct 

In [6]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/icpr2020dfdc/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/icpr2020dfdc/package.json'
npm WARN icpr2020dfdc No description
npm WARN icpr2020dfdc No repository field.
npm WARN icpr2020dfdc No README data
npm WARN icpr2020dfdc No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.758s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [7]:
!streamlit run app.py &>/content/logs.txt &

**O número do IP a seguir será necessário para colar no campo correspondente para acessar o APP**

In [8]:
!wget -q -O - ipv4.icanhazip.com

34.125.182.49


**Cole o IP no espaço 'Endpoint IP', e clique em 'Click to Submit'**

In [9]:
!npx localtunnel --port 8501

npx: installed 22 in 2.685s
your url is: https://tiny-monkeys-vanish.loca.lt
^C
